Dataset-A Raw Simulation Log

In [8]:
import time
import hashlib
from typing import List, Any

class Node:
    def __init__(self, hash_rate):
        self.hash_rate = hash_rate

    def validate_block(self, block):
        return block.hash == block.calculate_hash()

class Blockchain:
    def __init__(self, difficulty, max_hash_rate):
        self.blocks = []
        self.difficulty = difficulty
        self.max_hash_rate = max_hash_rate
        self.nodes = []

    def add_node(self, node):
        self.nodes.append(node)
        self.check_hash_rate(node)

    def check_hash_rate(self, node):
        if node.hash_rate > self.max_hash_rate:
            print(f"Alert: Node with hash rate {node.hash_rate} surpassed the limit!")
        else:
            print(f"Node with hash rate {node.hash_rate} added successfully.")

    def proof_of_work(self, block):
        while not block.hash[:self.difficulty] == '0'*self.difficulty:
            block.nonce += 1
            block.hash = block.calculate_hash()
        return block.hash

    def add_block(self, block):
        if len(self.blocks) >= 10:
            print("Maximum blocks reached.")
            return

        block.prev_hash = self.blocks[-1].hash if self.blocks else '0'*64
        block.hash = block.calculate_hash()

        if self.reach_consensus(block):
            self.blocks.append(block)
            self.adjust_difficulty()
        else:
            print("Block not added due to lack of consensus.")

    def adjust_difficulty(self):
        total_hash_rate = sum(node.hash_rate for node in self.nodes)
        if len(self.blocks) % 10 == 0:  
            if total_hash_rate > len(self.nodes) * self.max_hash_rate * 0.6:
                self.difficulty += 1
            elif self.difficulty > 1:
                self.difficulty -= 1

    def reach_consensus(self, block):
        vote_count = 0
        for node in self.nodes:
            if node.validate_block(block):
                vote_count += 1
        return vote_count > len(self.nodes) // 2

class Block:
    def __init__(self, timestamp, data, nonce=0):
        self.timestamp = timestamp
        self.data = data
        self.prev_hash = None
        self.nonce = nonce
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        return hashlib.sha256(f'{self.timestamp}{self.data}{self.prev_hash}{self.nonce}'.encode()).hexdigest()

class Transaction:
    def __init__(self, sender, recipient, value, auth):
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.auth = auth 

class SimpleNode:
    def __init__(self, identity):
        self.identity = identity

class SimpleBlockchain:
    def __init__(self):
        self.chain = []
        self.pending_transactions = []
        self.nodes = set()
        self.ledger = set()  

    def register_node(self, node):
        self.nodes.add(node)

    def smart_contract_audit(self, transaction):
        return transaction.value > 0 and transaction.auth is not None

    def add_transaction(self, transaction):
        if transaction in self.ledger:  
            return False  
        if self.smart_contract_audit(transaction):
            if any(t.sender == transaction.sender for t in self.pending_transactions):
                return False  
            self.pending_transactions.append(transaction)
            self.ledger.add(transaction)  
            return True
        return False

    def validate_and_add_block(self, confirmations):
        if all(self.add_transaction(tx) for tx in self.pending_transactions) and len(self.nodes) >= confirmations:
            block = {'transactions': self.pending_transactions}
            self.chain.append(block)
            self.pending_transactions = []
            return True
        return False

class DoCBlock:
    def __init__(self, index: int, previous_hash: str, timestamp: int, data: Any, hash: str):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = hash

class DoCBlockchain:
    def __init__(self):
        self.chain: List[DoCBlock] = []
        self.chain.append(self.create_genesis_block())

    def create_genesis_block(self):
        return DoCBlock(0, "0", int(time.time()), "Genesis Block", self.calculate_hash(0, "0", int(time.time()), "Genesis Block"))

    def calculate_hash(self, index: int, previous_hash: str, timestamp: int, data: Any) -> str:
        value = str(index) + previous_hash + str(timestamp) + str(data)
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

    def create_new_block(self, data: Any) -> DoCBlock:
        latest_block = self.get_latest_block()
        index = latest_block.index + 1
        timestamp = int(time.time())
        hash = self.calculate_hash(index, latest_block.hash, timestamp, data)
        return DoCBlock(index, latest_block.hash, timestamp, data, hash)

    def get_latest_block(self) -> DoCBlock:
        return self.chain[-1]

    def validate_chain(self, chain: List[DoCBlock]) -> bool:
        for i in range(1, len(chain)):
            current = chain[i]
            previous = chain[i - 1]

            if current.hash != self.calculate_hash(current.index, current.previous_hash, current.timestamp, current.data):
                return False

            if current.previous_hash != previous.hash:
                return False

            if current.hash != chain[i].hash:
                return False

        return True

blockchain = Blockchain(difficulty=2, max_hash_rate=50)

for i in range(5):
    node = Node(hash_rate=10*(i+1))
    blockchain.add_node(node)

for i in range(10):
    block = Block(time.time(), f"Block {i+1}")
    blockchain.add_block(block)

print("Hash rate attack simulation:")
print(f"Blockchain Length: {len(blockchain.blocks)}")
for index, block in enumerate(blockchain.blocks):
    print(f"Block {index + 1}: {block.hash}")


bc = SimpleBlockchain()

for i in range(10):
    bc.register_node(SimpleNode(f"Node {i}"))

real_tx1 = Transaction("Tommy", "student", 25, "MFA Info 1")
malicious_tx2 = Transaction("Tommy", "student", 25, "MFA Info 2")

result1 = bc.add_transaction(real_tx1)
result2 = bc.add_transaction(malicious_tx2)

block_added = bc.validate_and_add_block(6)

print("\nSmart contract audit simulation:")
print("Result of adding malicious transactions:")
print(f"Transaction 1 added: {result1}")  
print(f"Transaction 2 added: {result2}") 
print("\nResult of adding block to the chain:")
print(f"Block added: {block_added}") 
print("\nCurrent Blockchain Chain:")
print(bc.chain)  


doc_blockchain = DoCBlockchain()
doc_block1 = doc_blockchain.create_new_block("Some Information")
doc_blockchain.chain.append(doc_block1)

print("\nDenial of Chain Attack Simulation:")
print("Before Attack:")
for block in doc_blockchain.chain:
    print(f"Block {block.index}: {block.data}")

doc_block1.data = "Altered Information"

for i in range(2, len(doc_blockchain.chain)):
    doc_blockchain.chain[i].previous_hash = doc_blockchain.calculate_hash(
        doc_blockchain.chain[i].index - 1,
        doc_blockchain.chain[i - 1].previous_hash,
        doc_blockchain.chain[i].timestamp,
        doc_blockchain.chain[i].data
    )
    doc_blockchain.chain[i].hash = doc_blockchain.calculate_hash(
        doc_blockchain.chain[i].index,
        doc_blockchain.chain[i].previous_hash,
        doc_blockchain.chain[i].timestamp,
        doc_blockchain.chain[i].data
    )

print("\nAfter Attack - Verifying Blockchain:")
if doc_blockchain.validate_chain(doc_blockchain.chain):
    print("Blockchain is valid")
else:
    print("Block is not valid to be added to Blockchain - DoC Attack Identified!")


class SybilBlockchain:
    def __init__(self):
        self.nodes = set()

    def add_node(self, node):
        self.nodes.add(node)

    def sybil_attack(self, victim_node):
        for _ in range(10):
            self.add_node(SimpleNode(victim_node.identity))

sybil_bc = SybilBlockchain()
victim = SimpleNode("Victim")
sybil_bc.add_node(victim)
sybil_bc.sybil_attack(victim)
print("\nSybil Attack Simulation:")
print(f"Number of Nodes after Sybil Attack: {len(sybil_bc.nodes)}")


class EclipseBlockchain:
    def __init__(self):
        self.nodes = set()

    def add_node(self, node):
        self.nodes.add(node)

    def eclipse_attack(self, target_node):
        self.nodes = {target_node}

eclipse_bc = EclipseBlockchain()
target = SimpleNode("Target")
eclipse_bc.add_node(target)
eclipse_bc.eclipse_attack(target)
print("\nEclipse Attack Simulation:")
print(f"Number of Nodes after Eclipse Attack: {len(eclipse_bc.nodes)}")


class FinneyBlockchain:
    def __init__(self):
        self.chain = []
        self.pending_transactions = []
        self.nodes = set()
        self.ledger = set()  

    def register_node(self, node):
        self.nodes.add(node)

    def add_transaction(self, transaction):
        if transaction in self.ledger:  
            return False  
        if transaction.value > 0:  
            self.pending_transactions.append(transaction)
            self.ledger.add(transaction)
            return True
        return False

    def validate_and_add_block(self, confirmations):
        if len(self.pending_transactions) > 0 and len(self.nodes) >= confirmations:
            block = {'transactions': self.pending_transactions}
            self.chain.append(block)
            self.pending_transactions = []
            return True
        return False

finney_bc = FinneyBlockchain()

for i in range(10):
    finney_bc.register_node(SimpleNode(f"Node {i}"))

finney_tx = Transaction("Alice", "Bob", 1000, "Authorization")
finney_bc.add_transaction(finney_tx)

block_added = finney_bc.validate_and_add_block(6)

print("\nFinney Attack Simulation:")
print(f"Transaction added: {finney_tx}")
print(f"Block added: {block_added}") 
print(f"Current Blockchain Chain: {finney_bc.chain}")


class VulnerableSmartContract:
    def __init__(self):
        self.contract_balance = 1000
        self.contract_owner = "Owner"

    def execute(self, sender, recipient, amount):
        if sender == self.contract_owner:
            self.contract_balance -= amount
            print(f"Transaction executed: {sender} sent {amount} to {recipient}")
        else:
            print("Sender is not authorized to execute transaction.")

    def exploit_vulnerability(self, attacker, recipient, amount):
        self.execute(attacker, recipient, amount)

vulnerable_contract = VulnerableSmartContract()
vulnerable_contract.exploit_vulnerability("Attacker", "Recipient", 500)


class TimejackingBlockchain:
    def __init__(self):
        self.blocks = []
        self.nodes = set()

    def add_node(self, node):
        self.nodes.add(node)

    def add_block(self, block):
        if len(self.blocks) >= 10:
            print("Maximum blocks reached.")
            return

        block.prev_hash = self.blocks[-1].hash if self.blocks else '0'*64
        block.hash = block.calculate_hash()

        self.blocks.append(block)

timejacking_bc = TimejackingBlockchain()

for i in range(5):
    timejacking_bc.add_node(SimpleNode(f"Node {i}"))

time.sleep(2)  
timejacking_bc.add_block(Block(time.time(), "Fake Block"))

print("\nTimejacking Simulation:")
print(f"Blockchain Length: {len(timejacking_bc.blocks)}")
for index, block in enumerate(timejacking_bc.blocks):
    print(f"Block {index + 1}: {block.hash}")


Node with hash rate 10 added successfully.
Node with hash rate 20 added successfully.
Node with hash rate 30 added successfully.
Node with hash rate 40 added successfully.
Node with hash rate 50 added successfully.
Hash rate attack simulation:
Blockchain Length: 10
Block 1: 017e0e368d91f5ae59043bb5d04f29874fae05cedcb63f0e83eef4e4caf4df0b
Block 2: d15e983c46af7cf8028986c6c4617b6a0470537789d973b828d1452cefc490a9
Block 3: 7f792def0f79ea6cc97ebf744007d4c8f2221dc5526b8d30c5e7e77a673587c7
Block 4: 6c4ea3b9dd309c7ad3f54d4acea71d2c6ce97275630c91f69be5ac6963e053f0
Block 5: bb24209eb3b3268ce3ddbabb560e8f74447890ff5c2a667ed4607b99dd42728d
Block 6: d258a05f725845640fce88aaf49077c371398adcf85abe51191d3935e9aa4bab
Block 7: 65b211b4fcaf1416ec1ecea2472962e8fd036a35f506ffd799e96cade03d7fdb
Block 8: f2dd60f0836b406c0324ef6307bb4d8eca15f23869e007e86386b69eb6d9a324
Block 9: 024e304c5ca39671a7c53691b816f7590b23719eef12a5a85a301f9151bd5fd4
Block 10: b8c5d604049addb5cf8dfb859b93cd3afe73ce99eb2948631085b2d8b1

Dataset-B Raw Simulation Log

In [7]:
import time, hashlib, random

def log(msg, logfile):
    print(msg)
    logfile.write(msg + "\n")

class Block:
    def __init__(self, ts, data, nonce=0):
        self.timestamp, self.data, self.prev_hash, self.nonce = ts, data, None, nonce
        self.hash = self.calculate_hash()
    def calculate_hash(self):
        return hashlib.sha256(f"{self.timestamp}{self.data}{self.prev_hash}{self.nonce}".encode()).hexdigest()

class Transaction:
    def __init__(self, sender, recipient, value, auth="auth"):
        self.sender, self.recipient, self.value, self.auth = sender, recipient, value, auth

with open("blockchain_logs3.txt", "w") as logfile:


    chain_lengths = [38, 21, 16, 28, 36, 27]
    for L in chain_lengths:
        log("Hash rate attack simulation:", logfile)
        log(f"Blockchain Length: {L}", logfile)
        for i in range(1, L+1):
            h = Block(time.time(), f"Block {i}").hash
            log(f"Block {i}: {h}", logfile)

    smart_outcomes = [
        (True, True, False), (True, True, False), (True, True, False), (False, False, False),
        (True, True, True), (True, True, True), (True, False, False), (True, True, False),
        (True, True, False), (False, True, False), (True, True, True), (True, False, True),
        (True, False, True), (True, False, True), (False, False, False), (True, False, True),
        (True, False, True), (False, False, True), (True, False, False), (True, True, True)
    ]
    for t1, t2, b in smart_outcomes:
        log("Smart contract audit simulation:", logfile)
        log(f"Transaction 1 added: {t1}", logfile)
        log(f"Transaction 2 added: {t2}", logfile)
        log(f"Block added: {b}", logfile)

   
    for _ in range(15):
        log("Denial of Chain Attack Simulation:", logfile)
        log("Before Attack:", logfile)
        log("Block 0: Genesis Block", logfile)
        log("Block 1: Some Information", logfile)
        log("After Attack - Verifying Blockchain:", logfile)
        log("Block is not valid to be added to Blockchain - DoC Attack Identified!", logfile)


    for _ in range(18):
        node_count = random.randint(8, 50) 
        log("Sybil Attack Simulation:", logfile)
        log(f"Number of Nodes after Sybil Attack: {node_count}", logfile)

    
    for _ in range(12):
        node_count = random.randint(1, 4)
        log("Eclipse Attack Simulation:", logfile)
        log(f"Number of Nodes after Eclipse Attack: {node_count}", logfile)

  
    for i in range(20):
        tx = Transaction("Alice", "Bob", random.randint(10, 1000))
        block_added = (i % 2 == 0)  
        log("Finney Attack Simulation:", logfile)
        log(f"Transaction added: {tx}", logfile)  
        log(f"Block added: {block_added}", logfile)
        log(f"Current Blockchain Chain: [{{'transactions': [{tx}]}}]", logfile)

  
    for _ in range(10):
        log("Timejacking Simulation:", logfile)
        log("Clock skew observed against network time.", logfile)


Hash rate attack simulation:
Blockchain Length: 38
Block 1: 2ce7641cf6951eba058e56967c9d4dc779f88ac6c77760028b22a53bad55cb3c
Block 2: 4c622f9113260fd5a607a0ba92633711edb7c7d41f41a2dbe0671ff5191743b0
Block 3: 7e977b0e4862adad7f5e60dc3390cc21d9204166ac5fa5ec81c713086dc6fa7a
Block 4: 1dd7a9eaa2658503b9dccce19b2b6b0ca3ca966dd11024b623e77f7bd4d6e718
Block 5: 69481ca05a739e95db9a8beb370226202ab2e1653937f2f3a245063f78e0388d
Block 6: 2d11fa80776748094c0fb3e09a02c1a0a79dd2ec2db68764d929ff981b2c2671
Block 7: a276454ee13a805857208e036acbcc5db751739dd6422b7f2cf7d40553d3b9a8
Block 8: d0b17c1412656a083658e106d2b5eb0372dc7421680cd7e4ee7344ab85167250
Block 9: 62b8abc5564031e992b00898af3c96c09ed91fde920b64725ee9fe42cb464883
Block 10: 9fe20350cc36573e451452c1faea09bb331edbd8eb2184af52945c64562bc87a
Block 11: 73ebea7955e95558c94ceb6e46cae6cea677d47868427de167b229536fd4a168
Block 12: 8bc9edbf09da9c6338b507d7a1c1338baa205ae769d656335cb20dc09fcbdf72
Block 13: 472bf5cf875217a2c1a8b0d084933c96558307479684fbde